In [ ]:
import json, gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from pathlib import Path

data = Path('data')
pd.set_option('display.max_rows', None)

# Load the json file
with gzip.open(data / 'franchises.json.gz', 'rt') as f:
	franchises = json.load(f)

# Convert the franchises list to a dataframe
df = pd.DataFrame(franchises)

# Show the dataframe
df = df.sort_values(by=['my_score'], ascending=False)
df

In [ ]:
# Create the figure and subplots for histograms
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Histograms of 'score' and 'my_score'
sns.histplot(df['score'], kde=True, bins=30, ax=axs[0])
axs[0].set_title('Score Distribution of Franchises')
axs[0].set_xlabel('Score')
axs[0].set_ylabel('Density')

sns.histplot(df['my_score'], kde=True, bins=30, ax=axs[1])
axs[1].set_title('My Score Distribution of Franchises')
axs[1].set_xlabel('My Score')
axs[1].set_ylabel('Density')

plt.tight_layout()
plt.show()

# Helper function for generating box plots
def box_plot(data, x, y, title, xlabel, ylabel):
    if data[x].notna().any():  # Check if there are any non-null values
        plt.figure(figsize=(14, 6))
        sns.boxplot(x=x, y=y, data=data, order=order)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.xticks(rotation=90)
        plt.show()

# Unnest 'genres', 'themes', and 'studios' fields and create box plots
for field in ['genres', 'themes', 'studios']:
    df_unnested = df.explode(field)
    
    # Sort by descending average my_score
    order = df_unnested.groupby(field)['my_score'].median().sort_values(ascending=False).index
    
    box_plot(df_unnested, field, 'my_score', f'My Score Distribution by {field.capitalize()}', field.capitalize(), 'My Score')

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

def draw_co_occurrence_matrix(data, title):
    """Draw a co-occurrence matrix with a title and masks the upper triangle."""
    co_occurrence_matrix = np.dot(data.transpose(), data)
    co_occurrence_df = pd.DataFrame(co_occurrence_matrix, index=data.columns, columns=data.columns)
    mask = np.triu(np.ones_like(co_occurrence_df, dtype=bool))
    plt.figure(figsize=(10, 10))
    sns.heatmap(co_occurrence_df, mask=mask, cmap='coolwarm', center=0, annot=False, fmt='d')
    plt.title(title)
    plt.show()

# Initialize MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

# Extract and one-hot encode genres and themes
for feature in ['genres', 'themes']:
    feature_list = [franchise[feature] for franchise in franchises]
    feature_encoded = mlb.fit_transform(feature_list)
    df_feature = pd.DataFrame(feature_encoded, columns=mlb.classes_)
    draw_co_occurrence_matrix(df_feature, f'{feature.capitalize()} Co-occurrence Matrix')

# Most unpopular opinions

In [ ]:
df['score_difference'] = df['my_score'] - df['score']
df['score_difference_abs'] = (df['score_difference']).abs()
df_sorted_abs = df.sort_values(by='score_difference_abs', ascending=False)
df_sorted_abs[['title', 'score', 'my_score', 'score_difference']]

# My opinion vs the world's

In [ ]:
# Define the color thresholds using List of Tuples
color_thresholds = [
    (0.5, 'green'),
    (1.5, 'orange'),
]

# Define a function to get color based on score difference and the List of Tuples representation
def get_color(score_diff, thresholds):
    for threshold, color in thresholds:
        if score_diff <= threshold:
            return color
    return 'red'

# Apply the function to the dataframe
df['Color'] = df['score_difference_abs'].apply(lambda x: get_color(x, color_thresholds))

# Using the provided code to plot the scatter plot
ax = df.plot.scatter(x='my_score', y='score', c=df['Color'], figsize=(10, 10), alpha=0.7, s=50, edgecolor='k')

# y=x guide line
ax.plot([0, 10], [0, 10], 'k--', linewidth=0.5)

plt.title('My Score vs Crowd Score')
plt.xlabel('My Score')
plt.ylabel('Crowd Score')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.xlim(0, 10)
plt.ylim(0, 10)
plt.show()